Code Lab 2: PatchCamelyon (PCAM) Histopathological Cancer Detection 
===

## The Problem
PCam an image classification dataset based off of one of the Camelyon16 tasks where 96x96, 3-color channel image patches are extracted from whole-slide images (WSI).  These slides were extracted from histopathologic scans of lymph node sections in order to determine if the lymph nodes contain metastases (cancer cells).  From the Camelyon16 web page:
>  This task has a high clinical relevance but requires large amounts of reading time from pathologists. Therefore, a successful solution would hold great promise to reduce the workload of the pathologists while at the same time reduce the subjectivity in diagnosis. [1](https://camelyon16.grand-challenge.org/)

![pcam-cover](https://github.com/basveeling/pcam/raw/master/pcam.jpg)

In this lab, we will explore transfer learning and explainable AI (XAI) techniques on the PCam dataset.

## Imports

We will start off using a similar setup to Code Lab 1.

In [4]:
from __future__ import print_function, division
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import os
import pandas as pd
from IPython.display import display, HTML

We will continue to use the Eager Execution API for Tensorflow 1.13.  Let us make sure that it is enabled and that the GPU is visible.

In [6]:
import tensorflow as tf
tf.enable_eager_execution()

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print('Tensorflow version: ',tf.__version__)
print('Is GPU available: %s' % str(tf.test.is_gpu_available()))
print('Is Eager Execution enabled?: %s' % str(tf.executing_eagerly()))

Tensorflow version:  1.13.1
Is GPU available: False
Is Eager Execution enabled?: True


## Dataset

### About Pcam

PCam is contained in three data splits:
    1. Training: 262K samples
    2. Validation: 33K
    3. Testing: 33K

The patches are labeled 'positive' for containing a metastases if there was at least one pixel of segment in the central 32x32 region of the patch from the segmentation of the WSI that the patch was extracted from.  The dataset was constructed to be evenly balanced with 50/50 'present'/'not present'.

### Load the dataset